# WELCOME

here we have some example to access databases from the same extensions we wrote on `rapydo` for Flask

## create an object

this will be needed in the examples

In [1]:
from pytz import utc
from datetime import datetime
now = datetime.now(utc)

In [2]:
example_dict = {
    'name': "Test", 
    'truth': True, 
    'description': "Hello World", 
    'created': now
}

---

## MONGODB

### connect to mongo

In [3]:
from proof.apis.services import mongo

In [4]:
client, odm, db_name = mongo()

# client -> a normal pymongo connection
# odm -> a link to use ODM (like ORM) models 
# db_name -> the database name for the default db in mongo

print("Current db: %s" % db_name)

Connected to service: mongo
Current db: proof


### using ODM

For more models write one into `projects/proof/backend/models/mongo.py`

They are automatically injected in the ODM object!

In [5]:
element = odm.Examples(**example_dict).save()
# now this element is saved inside the db
element

Examples(name='Test', truth=True, description='Hello World', created=datetime.datetime(2018, 7, 20, 22, 58, 34, 282552, tzinfo=<UTC>), value=1)

### check with a GUI

If you launch the interface from command line:

`rapydo --mode notebook interfaces mongo` and then access:

http://localhost:8081/db/proof/examples

---

## POSTGRESQL

### connect to postgres

In [6]:
from proof.apis.services import postgres
app, orm, db_name = postgres()

# app -> a flask mock app to use sqlalchemy as you would do in the flask app
# orm -> a link to use ORM models 
# db_name -> the database name for the default db in postgres/sqlalchemy

print("Current db: %s" % db_name)

Connected to service: sqlalchemy
Current db: proof


### use ORM

For more models write one into `projects/proof/backend/models/sqlalchemy.py`

They are automatically injected in the ORM object!

In [7]:
with app.app_context():
    orm.Examples.query.delete() # truncate table to avoid primary keys clashing
    example = orm.Examples(**example_dict)
    orm.session.add(example)
    orm.session.commit()
    
# NOTE: you need to use the Flask app context 
# because `sqlalchemy` in rapydo is the one from Flask-Sqlalchemy plugin    

example

<Examples 11>

### check with a GUI

If you launch the interface from command line:

`rapydo --mode notebook interfaces sqlalchemy` and then access:

http://localhost:8082/?pgsql=psqldb.dockerized.io&username=admin&db=proof&ns=public&select=examples

the password is:

In [8]:
! env | grep ALC | grep PASS

ALCHEMY_PASSWORD=somenicelongpasswordtowritedown


### normal SQL queries

In [9]:
with app.app_context():
    from sqlalchemy import text
    sql = text('SELECT * from examples')

    result = orm.engine.execute(sql)
    for row in result:
        print(row)

(11, 'Test', True, 'Hello World', datetime.datetime(2018, 7, 20, 22, 58, 34, 282552), 1)


---

# THE END `:)`